In [1]:
from __future__ import division

from models import *
from utils.utils import *
from utils.datasets import *
from utils.parse_config import *

import os
import sys
import time
import datetime
import argparse

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

class OPT():
    def __init__(self):
        self.epochs=1
        self.image_folder="data/samples"
        self.batch_size=8
        self.model_config_path="config/yolov3_train768_anchor.cfg"
        self.data_config_path="config/coco.data"
        self.weights_path="weights/yolov3.weights"
        self.class_path="data/AI.names"
        self.conf_thres=0.8
        self.nms_thres=0.4
        self.n_cpu=0
        self.img_size=864
        self.checkpoint_interval=1
        self.checkpoint_dir="weights/768"
        self.use_cuda=True
        
opt=OPT()
print(opt)


cuda = torch.cuda.is_available() and opt.use_cuda

os.makedirs("output", exist_ok=True)
os.makedirs("checkpoints", exist_ok=True)

classes = load_classes(opt.class_path)

# Get data configuration
data_config = parse_data_config(opt.data_config_path)
train_path = data_config["train"]

# Get hyper parameters
hyperparams = parse_model_config(opt.model_config_path)[0]
learning_rate = float(hyperparams["learning_rate"])
momentum = float(hyperparams["momentum"])
decay = float(hyperparams["decay"])
burn_in = int(hyperparams["burn_in"])

# Initiate model
model = Darknet(opt.model_config_path,opt.img_size)
model.load_weights(opt.weights_path)
#model.apply(weights_init_normal)
model.module_list[105]=nn.Sequential(nn.Conv2d(256, 45, kernel_size=(1, 1), stride=(1, 1)))
model.module_list[93]=nn.Sequential(nn.Conv2d(512, 45, kernel_size=(1, 1), stride=(1, 1)))
model.module_list[81]=nn.Sequential(nn.Conv2d(1024, 45, kernel_size=(1, 1), stride=(1, 1)))
model=model_filter(model)
if cuda:
    model = model.cuda()

model.train()

# Get dataloader
dataloader = torch.utils.data.DataLoader(
    TrainDataset(train_path), batch_size=opt.batch_size, shuffle=False)

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
for epoch in range(opt.epochs):
    for batch_i, (_, imgs, targets) in enumerate(dataloader):
        imgs = Variable(imgs.type(Tensor))
        targets = Variable(targets.type(Tensor), requires_grad=False)
        
        multiscale_training(model,imgs.shape[2])
        optimizer.zero_grad()
        
        loss = model(imgs, targets)
        
        loss.backward()
        optimizer.step()
        if batch_i%6==0:
            print(
                "[Epoch %d/%d, Batch %d/%d] [Losses: x %f, y %f, w %f, h %f, conf %f, cls %f, total %f, recall: %.5f, precision: %.5f,img_size: %d]"
                % (
                    epoch,
                    opt.epochs,
                    batch_i,
                    len(dataloader),
                    model.losses["x"],
                    model.losses["y"],
                    model.losses["w"],
                    model.losses["h"],
                    model.losses["conf"],
                    model.losses["cls"],
                    loss.item(),
                    model.losses["recall"],
                    model.losses["precision"],
                    model.img_size,
                )
            )
            
            model.seen += imgs.size(0)
    
    if epoch % opt.checkpoint_interval == 0:
        model.save_weights("%s/%d.weights" % (opt.checkpoint_dir, epoch))
#        if batch_i % 2900 == 0:
#            model.save_weights("%s/%d.weights" % (opt.checkpoint_dir, batch_i))



/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/local/lib/python3.5/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


[Epoch 0/1, Batch 0/8733] [Losses: x 0.004624, y 0.004547, w 0.010421, h 0.006260, conf 0.684738, cls 0.004276, total 0.198848, recall: 0.16931, precision: 0.00000,img_size: 672]
[Epoch 0/1, Batch 6/8733] [Losses: x 0.007453, y 0.007512, w 0.004308, h 0.004528, conf 0.205609, cls 0.003434, total 0.058570, recall: 0.50820, precision: 0.00000,img_size: 736]
[Epoch 0/1, Batch 12/8733] [Losses: x 0.006722, y 0.006758, w 0.001940, h 0.002028, conf 0.077221, cls 0.001911, total 0.026170, recall: 0.64691, precision: 0.00000,img_size: 800]
[Epoch 0/1, Batch 18/8733] [Losses: x 0.006444, y 0.006452, w 0.001620, h 0.001750, conf 0.023199, cls 0.002357, total 0.013673, recall: 0.66667, precision: 0.00000,img_size: 864]
[Epoch 0/1, Batch 24/8733] [Losses: x 0.007808, y 0.007445, w 0.002680, h 0.001935, conf 0.036066, cls 0.002121, total 0.015545, recall: 0.54878, precision: 0.00000,img_size: 608]
[Epoch 0/1, Batch 30/8733] [Losses: x 0.012038, y 0.011352, w 0.003410, h 0.001930, conf 0.023594, cls

[Epoch 0/1, Batch 276/8733] [Losses: x 0.007566, y 0.006716, w 0.001305, h 0.001119, conf 0.003024, cls 0.001372, total 0.005534, recall: 0.86288, precision: 0.00000,img_size: 736]
[Epoch 0/1, Batch 282/8733] [Losses: x 0.006225, y 0.005664, w 0.001206, h 0.000942, conf 0.009920, cls 0.001105, total 0.006698, recall: 0.82353, precision: 0.00000,img_size: 800]
[Epoch 0/1, Batch 288/8733] [Losses: x 0.006861, y 0.006211, w 0.001226, h 0.000997, conf 0.007139, cls 0.001087, total 0.006183, recall: 0.87647, precision: 0.00000,img_size: 864]
[Epoch 0/1, Batch 294/8733] [Losses: x 0.010704, y 0.010367, w 0.001937, h 0.001880, conf 0.010435, cls 0.002258, total 0.009934, recall: 0.70552, precision: 0.00000,img_size: 608]
[Epoch 0/1, Batch 300/8733] [Losses: x 0.006677, y 0.006185, w 0.002306, h 0.001544, conf 0.014805, cls 0.001090, total 0.008448, recall: 0.79710, precision: 0.00000,img_size: 672]
[Epoch 0/1, Batch 306/8733] [Losses: x 0.007685, y 0.007254, w 0.001300, h 0.001176, conf 0.002

[Epoch 0/1, Batch 552/8733] [Losses: x 0.005380, y 0.004834, w 0.001014, h 0.000698, conf 0.003424, cls 0.000675, total 0.004239, recall: 0.82704, precision: 0.00000,img_size: 800]
[Epoch 0/1, Batch 558/8733] [Losses: x 0.005600, y 0.005387, w 0.000969, h 0.001187, conf 0.002253, cls 0.000675, total 0.004343, recall: 0.79851, precision: 0.00000,img_size: 864]
[Epoch 0/1, Batch 564/8733] [Losses: x 0.011470, y 0.010780, w 0.002469, h 0.001547, conf 0.009431, cls 0.001467, total 0.009764, recall: 0.77546, precision: 0.00000,img_size: 608]
[Epoch 0/1, Batch 570/8733] [Losses: x 0.009619, y 0.009028, w 0.001898, h 0.001224, conf 0.005927, cls 0.002046, total 0.007810, recall: 0.83821, precision: 0.00000,img_size: 672]
[Epoch 0/1, Batch 576/8733] [Losses: x 0.006806, y 0.006003, w 0.001395, h 0.000800, conf 0.003174, cls 0.001066, total 0.005145, recall: 0.87534, precision: 0.00000,img_size: 736]
[Epoch 0/1, Batch 582/8733] [Losses: x 0.007167, y 0.006498, w 0.000866, h 0.000644, conf 0.005

[Epoch 0/1, Batch 828/8733] [Losses: x 0.004796, y 0.004174, w 0.000607, h 0.000584, conf 0.006593, cls 0.000653, total 0.004549, recall: 0.91111, precision: 0.00000,img_size: 864]
[Epoch 0/1, Batch 834/8733] [Losses: x 0.010049, y 0.008601, w 0.001993, h 0.001340, conf 0.006342, cls 0.000957, total 0.007767, recall: 0.80779, precision: 0.00000,img_size: 608]
[Epoch 0/1, Batch 840/8733] [Losses: x 0.009229, y 0.008525, w 0.001878, h 0.001340, conf 0.003874, cls 0.001188, total 0.006903, recall: 0.79762, precision: 0.00000,img_size: 672]
[Epoch 0/1, Batch 846/8733] [Losses: x 0.009598, y 0.009093, w 0.001381, h 0.001095, conf 0.012423, cls 0.001609, total 0.009150, recall: 0.85661, precision: 0.00000,img_size: 736]
[Epoch 0/1, Batch 852/8733] [Losses: x 0.005016, y 0.004749, w 0.000561, h 0.000815, conf 0.003574, cls 0.000836, total 0.004099, recall: 0.88679, precision: 0.00000,img_size: 800]
[Epoch 0/1, Batch 858/8733] [Losses: x 0.007299, y 0.006848, w 0.001141, h 0.000833, conf 0.003

...

[Epoch 0/1, Batch 7452/8733] [Losses: x 0.006821, y 0.006425, w 0.000807, h 0.000672, conf 0.004067, cls 0.000758, total 0.005130, recall: 0.90870, precision: 0.00000,img_size: 800]
[Epoch 0/1, Batch 7458/8733] [Losses: x 0.005643, y 0.005553, w 0.000868, h 0.000753, conf 0.003463, cls 0.000805, total 0.004462, recall: 0.90541, precision: 0.00000,img_size: 864]
[Epoch 0/1, Batch 7464/8733] [Losses: x 0.011059, y 0.010382, w 0.001740, h 0.000942, conf 0.004435, cls 0.001688, total 0.007936, recall: 0.87859, precision: 0.00000,img_size: 608]
[Epoch 0/1, Batch 7470/8733] [Losses: x 0.008179, y 0.007567, w 0.001204, h 0.000775, conf 0.002958, cls 0.001022, total 0.005714, recall: 0.86052, precision: 0.00000,img_size: 672]
[Epoch 0/1, Batch 7476/8733] [Losses: x 0.006883, y 0.006140, w 0.000640, h 0.000601, conf 0.004097, cls 0.000870, total 0.005103, recall: 0.90338, precision: 0.00000,img_size: 736]
[Epoch 0/1, Batch 7482/8733] [Losses: x 0.006730, y 0.006386, w 0.000933, h 0.000579, conf

[Epoch 0/1, Batch 7728/8733] [Losses: x 0.006599, y 0.006021, w 0.000805, h 0.000528, conf 0.006072, cls 0.000808, total 0.005441, recall: 0.90147, precision: 0.00000,img_size: 864]
[Epoch 0/1, Batch 7734/8733] [Losses: x 0.011870, y 0.009822, w 0.001638, h 0.001193, conf 0.002141, cls 0.001215, total 0.007506, recall: 0.85281, precision: 0.00000,img_size: 608]
[Epoch 0/1, Batch 7740/8733] [Losses: x 0.004900, y 0.004243, w 0.000456, h 0.000495, conf 0.006966, cls 0.000712, total 0.004624, recall: 0.89640, precision: 0.00000,img_size: 672]
[Epoch 0/1, Batch 7746/8733] [Losses: x 0.006576, y 0.006243, w 0.000810, h 0.000600, conf 0.004846, cls 0.000742, total 0.005191, recall: 0.88802, precision: 0.00000,img_size: 736]
[Epoch 0/1, Batch 7752/8733] [Losses: x 0.005867, y 0.005326, w 0.000512, h 0.000422, conf 0.002152, cls 0.000634, total 0.003948, recall: 0.92366, precision: 0.00000,img_size: 800]
[Epoch 0/1, Batch 7758/8733] [Losses: x 0.007479, y 0.007115, w 0.001099, h 0.000771, conf

[Epoch 0/1, Batch 8004/8733] [Losses: x 0.011572, y 0.010674, w 0.001853, h 0.001904, conf 0.003652, cls 0.001426, total 0.008264, recall: 0.88381, precision: 0.00000,img_size: 608]
[Epoch 0/1, Batch 8010/8733] [Losses: x 0.009136, y 0.008230, w 0.000887, h 0.000944, conf 0.003695, cls 0.001337, total 0.006394, recall: 0.89038, precision: 0.00000,img_size: 672]
[Epoch 0/1, Batch 8016/8733] [Losses: x 0.006888, y 0.006166, w 0.001117, h 0.000892, conf 0.009490, cls 0.000746, total 0.006589, recall: 0.89044, precision: 0.00000,img_size: 736]
[Epoch 0/1, Batch 8022/8733] [Losses: x 0.008259, y 0.007711, w 0.001196, h 0.000709, conf 0.002021, cls 0.001547, total 0.005640, recall: 0.90018, precision: 0.00000,img_size: 800]
[Epoch 0/1, Batch 8028/8733] [Losses: x 0.005531, y 0.005528, w 0.000769, h 0.000487, conf 0.005491, cls 0.000851, total 0.004885, recall: 0.87793, precision: 0.00000,img_size: 864]
[Epoch 0/1, Batch 8034/8733] [Losses: x 0.010515, y 0.009454, w 0.001557, h 0.001292, conf

[Epoch 0/1, Batch 8280/8733] [Losses: x 0.010308, y 0.010117, w 0.001805, h 0.001349, conf 0.002375, cls 0.001633, total 0.007245, recall: 0.85098, precision: 0.00000,img_size: 672]
[Epoch 0/1, Batch 8286/8733] [Losses: x 0.010271, y 0.009852, w 0.001780, h 0.001527, conf 0.003104, cls 0.001587, total 0.007380, recall: 0.87520, precision: 0.00000,img_size: 736]
[Epoch 0/1, Batch 8292/8733] [Losses: x 0.007679, y 0.007021, w 0.000986, h 0.000965, conf 0.006849, cls 0.000874, total 0.006348, recall: 0.90392, precision: 0.00000,img_size: 800]
[Epoch 0/1, Batch 8298/8733] [Losses: x 0.007338, y 0.007025, w 0.000919, h 0.000776, conf 0.001694, cls 0.001013, total 0.004968, recall: 0.89552, precision: 0.00000,img_size: 864]
[Epoch 0/1, Batch 8304/8733] [Losses: x 0.010938, y 0.010393, w 0.001451, h 0.001152, conf 0.017183, cls 0.002318, total 0.011399, recall: 0.85663, precision: 0.00000,img_size: 608]
[Epoch 0/1, Batch 8310/8733] [Losses: x 0.009428, y 0.008879, w 0.001966, h 0.001086, conf

[Epoch 0/1, Batch 8556/8733] [Losses: x 0.006256, y 0.005897, w 0.000730, h 0.000482, conf 0.002585, cls 0.000694, total 0.004417, recall: 0.89532, precision: 0.00000,img_size: 736]
[Epoch 0/1, Batch 8562/8733] [Losses: x 0.006473, y 0.006094, w 0.000599, h 0.000654, conf 0.006368, cls 0.000660, total 0.005437, recall: 0.91034, precision: 0.00000,img_size: 800]
[Epoch 0/1, Batch 8568/8733] [Losses: x 0.005612, y 0.004929, w 0.000527, h 0.000521, conf 0.004788, cls 0.000459, total 0.004390, recall: 0.94799, precision: 0.00000,img_size: 864]
[Epoch 0/1, Batch 8574/8733] [Losses: x 0.008341, y 0.007617, w 0.000689, h 0.000761, conf 0.000739, cls 0.000730, total 0.005033, recall: 0.90437, precision: 0.00000,img_size: 608]
[Epoch 0/1, Batch 8580/8733] [Losses: x 0.009574, y 0.008727, w 0.001136, h 0.001117, conf 0.006479, cls 0.001087, total 0.007346, recall: 0.89414, precision: 0.00000,img_size: 672]
[Epoch 0/1, Batch 8586/8733] [Losses: x 0.006705, y 0.006212, w 0.000813, h 0.000504, conf

In [ ]:
type(opt)

In [ ]:
 model.save_weights("%s/1.weights" % (opt.checkpoint_dir))